In [36]:
# The code was removed by Watson Studio for sharing.

# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 1: Collecting the data


Estimated time needed: **45** minutes


In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing\_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans.


## Objectives


In this lab, you will make a get request to the SpaceX API. You will also do some basic data wrangling and formating.

*   Request to the SpaceX API
*   Clean the requested data


***


## Import Libraries and Define Auxiliary Functions


We will import the following libraries into the lab


In [37]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)
# Setting to print all rows
pd.set_option('display.max_rows', None)

print('Libraries imported')

Libraries imported


#### Create Functions to extract data from the SpaceX API

From <code>rockets</code> endpoint extract the names.

In [38]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.


In [39]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
        response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
        Longitude.append(response['longitude'])
        Latitude.append(response['latitude'])
        LaunchSite.append(response['name'])

From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.


In [40]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])
        PayloadName.append(response['name'])
        try:
            PayloadCustomer.append(response['customers'][0])
        except:
            PayloadCustomer.append('None')

From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [41]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
                Status.append(response['status'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
                Statuts.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

From <code>landpads</code> get the Land Pads names:

In [42]:
def getLandingPad(data):
    for pad in data['LandingPad']:
        if pad is None:
            LandingPadName.append(pad)
        else:
            response = requests.get("https://api.spacexdata.com/v4/landpads/"+pad).json()
            LandingPadName.append(response['name'])

Create <code>URL</code> object for the request.

In [43]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

Create a <code>request</code> object.

In [44]:
results = requests.get(spacex_url).json()

Normalize the results.

In [45]:
df1 = pd.json_normalize(results)
print(df1.shape)
df1.head()

(150, 43)


,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings.ships,links.patch.small,links.patch.large,links.reddit.campaign,links.reddit.launch,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,[],https://images2.imgbox.com/3c/0e/T8iJcSN3_o.png,https://images2.imgbox.com/40/e3/GypSkayF_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'harmonic oscillation leading to premature engine shutdown'}]","Successful first stage burn and transition to second stage, maximum altitude 289 km, Premature engine shutdown at T+7 min 30 s, Failed to reach orbit, Failed to recover first stage",[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdaffd86e000604b32b,False,False,False,[],https://images2.imgbox.com/4f/e3/I0lkuJ2e_o.png,https://images2.imgbox.com/be/e7/iNqsqVYM_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-rocket-fails-reach-orbit.html,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'residual stage-1 thrust led to collision between stage 1 and stage 2'}]",Residual stage 1 thrust led to collision between stage 1 and stage 2,[],[],[],"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006eeb1e4]",5e9e4502f5090995de566f86,3,Trailblazer,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdbffd86e000604b32c,False,False,False,[],https://images2.imgbox.com/3d/86/cnu0pan8_o.png,https://images2.imgbox.com/4b/bd/d8UxLh4q_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1-flight-3-mission-summary,https://en.wikipedia.org/wiki/Trailblazer_(satellite),NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],"Ratsat was carried to orbit on the first successful orbital launch of any privately funded and developed, liquid-propelled carrier rocket, the SpaceX Falcon 1",[],[],[],[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,4,RatSat,2008-09-28T23:15:00.000Z,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': 

Filter the columns for thsi project.

In [46]:
df1 = df1[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

Keep only the rows in which the 'cores' column contains only one core listed.

In [47]:
df1 = df1[df1['cores'].map(len)==1]

Keep only the rows in which the 'payloads' column contains only one payload listed.

In [48]:
df1 = df1[df1['payloads'].map(len)==1]

Extract the dictionary out of the list in the 'cores' column.

In [49]:
df1['cores'] = df1['cores'].map(lambda x : x[0])

Extract the payload id out of the list in the 'payloads' column.

In [50]:
df1['payloads'] = df1['payloads'].map(lambda x : x[0])

Convert the 'date' column to <code>datetime</code> series, and extract only the date.

In [51]:
df1['date'] = pd.to_datetime(df1['date_utc']).dt.date

Create lists for the extracted from the <code>API</code> using the functions created.

In [52]:
BoosterVersion = []
PayloadMass = []
Orbit = []
PayloadName = []
PayloadCustomer = []
Flights = []
GridFins = []
Outcome = []
Reused = []
Legs = []
LandingPad = []
Block = []
Serial = []
Status = []
ReusedCount = []
LaunchSite = []
Longitude = []
Latitude = []

Call the function to get Booster Versions.

In [53]:
getBoosterVersion(df1)

Call the function to get the Payload data.

In [54]:
getPayloadData(df1)

Call the function to get the Launch Sites.

In [55]:
getLaunchSite(df1)

Call the function to get the Core data

In [56]:
getCoreData(df1)

Create dictionary with the lists created in the previous cell.

In [57]:
launch_dict = {'FlightNumber': list(df1['flight_number']),
'Date': list(df1['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'PayloadName':PayloadName,
'Customer':PayloadCustomer,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'Serial':Serial,
'Status':Status,
'ReusedCount':ReusedCount,
'LaunchSite':LaunchSite,
'Longitude':Longitude,
'Latitude':Latitude}

Create a <code>DataFrame</code> with the dictionary created in the previous cell

In [58]:
new_df = pd.DataFrame(launch_dict)
print(new_df.shape)
new_df.head()

(137, 20)


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,PayloadName,Customer,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Serial,Status,ReusedCount,LaunchSite,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.0,LEO,FalconSAT-2,DARPA,None None,1,False,False,False,None,NaN,Merlin1A,lost,0,Kwajalein Atoll,167.743129,9.047721
1,2,2007-03-21,Falcon 1,NaN,LEO,DemoSAT,DARPA,None None,1,False,False,False,None,NaN,Merlin2A,lost,0,Kwajalein Atoll,167.743129,9.047721
2,4,2008-09-28,Falcon 1,165.0,LEO,RatSat,SpaceX,None None,1,False,False,False,None,NaN,Merlin2C,lost,0,Kwajalein Atoll,167.743129,9.047721
3,5,2009-07-13,Falcon 1,200.0,LEO,RazakSAT,ATSB,None None,1,False,False,False,None,NaN,Merlin3C,lost,0,Kwajalein Atoll,167.743129,9.047721
4,6,2010-06-04,Falcon 9,NaN,LEO,Dragon Qualification Unit,SpaceX,None None,1,False,False,False,None,1.0,B0003,expended,0,CCSFS SLC 40,-80.577366,28.561857


Create a list for the 'LandingPad' to laod data the API.

In [59]:
LandingPadName = []

Call the function to get the Landing Pads names.

In [60]:
getLandingPad(new_df)

Replace the values in the 'LandingPad' column with data in the LandingPadName list.

In [61]:
new_df['LandingPad'] = LandingPadName

Filter the <code>DataFrame</code> to keep only Falcon 9 BoosterVersions

In [62]:
data_falcon9 = new_df[new_df['BoosterVersion']=='Falcon 9'].reset_index(drop=True).copy()
print(data_falcon9.shape)
data_falcon9.head()

(133, 20)


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,PayloadName,Customer,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Serial,Status,ReusedCount,LaunchSite,Longitude,Latitude
0,6,2010-06-04,Falcon 9,NaN,LEO,Dragon Qualification Unit,SpaceX,None None,1,False,False,False,None,1.0,B0003,expended,0,CCSFS SLC 40,-80.577366,28.561857
1,8,2012-05-22,Falcon 9,525.0,LEO,COTS Demo Flight 2,NASA(COTS),None None,1,False,False,False,None,1.0,B0005,lost,0,CCSFS SLC 40,-80.577366,28.561857
2,10,2013-03-01,Falcon 9,677.0,ISS,CRS-2,NASA (CRS),None None,1,False,False,False,None,1.0,B0007,lost,0,CCSFS SLC 40,-80.577366,28.561857
3,11,2013-09-29,Falcon 9,500.0,PO,CASSIOPE,MDA,False Ocean,1,False,False,False,None,1.0,B1003,lost,0,VAFB SLC 4E,-120.610829,34.632093
4,12,2013-12-03,Falcon 9,3170.0,GTO,SES-8,SES,None None,1,False,False,False,None,1.0,B1004,lost,0,CCSFS SLC 40,-80.577366,28.561857


Replace the NaN values in 'PayloadMass' column with the mean of all values in the column.

In [63]:
payload_mean = data_falcon9.loc[:,'PayloadMass'].mean().round()

data_falcon9.replace({'PayloadMass': np.nan}, payload_mean, inplace=True)
print(data_falcon9.shape)
data_falcon9.head()

(133, 20)


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,PayloadName,Customer,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Serial,Status,ReusedCount,LaunchSite,Longitude,Latitude
0,6,2010-06-04,Falcon 9,7457.0,LEO,Dragon Qualification Unit,SpaceX,None None,1,False,False,False,None,1.0,B0003,expended,0,CCSFS SLC 40,-80.577366,28.561857
1,8,2012-05-22,Falcon 9,525.0,LEO,COTS Demo Flight 2,NASA(COTS),None None,1,False,False,False,None,1.0,B0005,lost,0,CCSFS SLC 40,-80.577366,28.561857
2,10,2013-03-01,Falcon 9,677.0,ISS,CRS-2,NASA (CRS),None None,1,False,False,False,None,1.0,B0007,lost,0,CCSFS SLC 40,-80.577366,28.561857
3,11,2013-09-29,Falcon 9,500.0,PO,CASSIOPE,MDA,False Ocean,1,False,False,False,None,1.0,B1003,lost,0,VAFB SLC 4E,-120.610829,34.632093
4,12,2013-12-03,Falcon 9,3170.0,GTO,SES-8,SES,None None,1,False,False,False,None,1.0,B1004,lost,0,CCSFS SLC 40,-80.577366,28.561857


Reset the flight numbers in the 'FlightNumber' column.

In [64]:
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
print(data_falcon9.shape)
data_falcon9.head()

(133, 20)


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,PayloadName,Customer,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Serial,Status,ReusedCount,LaunchSite,Longitude,Latitude
0,1,2010-06-04,Falcon 9,7457.0,LEO,Dragon Qualification Unit,SpaceX,None None,1,False,False,False,None,1.0,B0003,expended,0,CCSFS SLC 40,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.0,LEO,COTS Demo Flight 2,NASA(COTS),None None,1,False,False,False,None,1.0,B0005,lost,0,CCSFS SLC 40,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.0,ISS,CRS-2,NASA (CRS),None None,1,False,False,False,None,1.0,B0007,lost,0,CCSFS SLC 40,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.0,PO,CASSIOPE,MDA,False Ocean,1,False,False,False,None,1.0,B1003,lost,0,VAFB SLC 4E,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.0,GTO,SES-8,SES,None None,1,False,False,False,None,1.0,B1004,lost,0,CCSFS SLC 40,-80.577366,28.561857


Create a duplicate colummn from 'Blocks' columns and name it 'Block_Version'.

In [65]:
data_falcon9['Block_Version'] = data_falcon9['Block']

Repalce the numbers in the 'Block_Version' column with there respective names.

In [66]:
data_falcon9['Block_Version'].replace({1.0:'v1.0', 2.0:'v1.1', 3.0:'FT', 4.0:'B4', 5.0:'B5'}, inplace=True)

Print the <code>DataFrame</code>.

In [67]:
print(data_falcon9.shape)
data_falcon9.head()

(133, 21)


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,PayloadName,Customer,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Serial,Status,ReusedCount,LaunchSite,Longitude,Latitude,Block_Version
0,1,2010-06-04,Falcon 9,7457.0,LEO,Dragon Qualification Unit,SpaceX,None None,1,False,False,False,None,1.0,B0003,expended,0,CCSFS SLC 40,-80.577366,28.561857,v1.0
1,2,2012-05-22,Falcon 9,525.0,LEO,COTS Demo Flight 2,NASA(COTS),None None,1,False,False,False,None,1.0,B0005,lost,0,CCSFS SLC 40,-80.577366,28.561857,v1.0
2,3,2013-03-01,Falcon 9,677.0,ISS,CRS-2,NASA (CRS),None None,1,False,False,False,None,1.0,B0007,lost,0,CCSFS SLC 40,-80.577366,28.561857,v1.0
3,4,2013-09-29,Falcon 9,500.0,PO,CASSIOPE,MDA,False Ocean,1,False,False,False,None,1.0,B1003,lost,0,VAFB SLC 4E,-120.610829,34.632093,v1.0
4,5,2013-12-03,Falcon 9,3170.0,GTO,SES-8,SES,None None,1,False,False,False,None,1.0,B1004,lost,0,CCSFS SLC 40,-80.577366,28.561857,v1.0


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range.


In [68]:
project.save_data(data=data_falcon9.to_csv(index=False), file_name ='spacex_api.csv', overwrite=True)

{'file_name': 'spacex_api.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'applieddatasciencecapstone-donotdelete-pr-jx5un88ldtszdu',
 'asset_id': 'e3d9174d-7142-4ae0-a172-c58e7c0fbe08'}

## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description                  |
| ----------------- | ------- | ---------- | ----------------------------------- |
| 2020-09-20        | 1.1     | Joseph     | get result each time you run        |
| 2020-09-20        | 1.1     | Azim       | Created Part 1 Lab using SpaceX API |
| 2020-09-20        | 1.0     | Joseph     | Modified Multiple Areas             |


Copyright © 2021 IBM Corporation. All rights reserved.
